In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from resnet import ResNet18
from resnetwm import ResNet18WM
import utils
from utils import*
import torch.utils.data as data
import torchvision
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import random
from torch.utils.data import TensorDataset
import mlp_private
# from mlp_private import MLP
from mlp import MLP
from dataset import BBoxDtaset
import cv2
from torch.utils.data.sampler import SubsetRandomSampler
import nni
from nni.compression.torch import LevelPruner, L1FilterPruner, FPGMPruner, SlimPruner, AGPPruner
from pruning_utils import*
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [2]:
#Hyperparameters
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 100
train_batch_size = 128
test_batch_size = 100
learning_rate = 0.001
momentum = 0.9
original_classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
watermark_classes = ('no key', 'top righ', 'top left', 'down right', 'dwon left')
identity_string = b'Y6588121SNAJEEBMOHARRAMSALIMJEBREEL02092020imageclassification_10_classes'
fake_identity_string = b'Y6588121NAJEEBMOHARRAMSALIMcrisesURV02092020classification10labels'
plagiarizer_identity_string = b'X7823579MRAMIJOSEPHAFFARcrisesURV2409022ResNet18classification10labels'
plagiarizer_fake_identity_string = b'X7823579WRAMIHAFFARJOSEP2509022ResNet18classificationofImage10labels'
key_size = 5
num_wm_samples = 25000
train_wm_black_box_epochs = 50
combined_original_model_epochs = 200
combined_private_model_epochs = 100
train_simultaneously_epochs = 100
dataset = 'cifar10'
containerdataset = 'stl10'

In [3]:
def updateBN(model):
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.weight.grad.data.add_(0.0001 * torch.sign(m.weight.data))  # L1

In [4]:
def train(model, device, train_loader, optimizer, criterion, sparse_bn=False):
    model.train()
    for batch_idx, (data, target, flags) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, _ = model(data)
        loss = criterion(output, target)
        loss.backward()
        # L1 regularization on BN layer
        if sparse_bn:
            updateBN(model)
        optimizer.step()
       
    print('Loss: {}'.format(loss.item()))


In [5]:
# Testing the original model on the original task
def test_original(model, criterion, test_loader, device):
   
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets, flags) in enumerate(test_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs, _ = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

            print('Testing: epoch: %d | loss: %.3f | Acc: %.3f' %(1, test_loss/(batch_idx+1), 100.*correct/total))
            
        return  test_loss/(batch_idx+1), 100.*correct/total

In [6]:
# Testing the private model on the wm task
def test_private(model, criterion, test_loader, device):
   
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets, flags) in enumerate(test_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                targets = targets.type(torch.LongTensor).to(device)
                _, outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

            print('Private model testing: epoch: %d | loss: %.3f | Acc: %.3f' %(1, test_loss/(batch_idx+1), 100.*correct/total))
            
            return  test_loss/(batch_idx+1), 100.*correct/total

In [3]:
combinedmodel = ResNet18WM().to(device)
combinedoptimizer = torch.optim.SGD(combinedmodel.parameters(), lr=0.001,
                      momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
checkpoint = torch.load('./Checkpoints/best_combined_fromscratch_cifar10.t7')
combinedmodel.load_state_dict(checkpoint['state_dict'])
combinedoptimizer.load_state_dict(checkpoint['optimizer'])
combinedstart_epoch = checkpoint['epoch']
combinedbest_acc = checkpoint['acc']
combinedbest_acc

92.07

In [8]:
trainset, testset, train_loader, test_loader = get_flagged_cifar10_dataset(100, 100)
# train_set, test_set = get_original_cifar10_dataset()
# train_loader = torch.utils.data.DataLoader(
#         train_set, batch_size=100, shuffle=True, num_workers=2)
# test_loader = torch.utils.data.DataLoader(
#         test_set, batch_size=100, shuffle=False, num_workers=2)
random_samples = np.random.choice(len(trainset), int(len(trainset)*0.3), replace = False)
sampled_trainset = torch.utils.data.Subset(trainset, random_samples)
finetune_train_loader = torch.utils.data.DataLoader(
        sampled_trainset, batch_size=128, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
trainsetwm, testsetwm = get_signed_dataset(identity_string, fake_identity_string, key_size, num_wm_samples
                                          , containerdataset=containerdataset)
wm_testloader = torch.utils.data.DataLoader(testsetwm, batch_size=10, shuffle=False, num_workers=1)

Files already downloaded and verified


In [11]:
pruning_rates = [0.1, 0.2, 0.3, 0.4, 0.45, 0.5]
for pr in pruning_rates:
    
    combinedmodel = ResNet18WM().to(device)
    combinedoptimizer = torch.optim.SGD(combinedmodel.parameters(), lr=0.001,
                          momentum=0.9, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    checkpoint = torch.load('./Checkpoints/best_combined_fromscratch_cifar10.t7')
    combinedmodel.load_state_dict(checkpoint['state_dict'])
    combinedstart_epoch = checkpoint['epoch']
    combinedbest_acc = checkpoint['acc']
    print('Marked model accuracy', combinedbest_acc)
    
    print('Pruning rate: {}%'.format(pr*100))
    
    config_list = [{ 'sparsity': pr, 'op_types': ['Conv2d'] }]

    pruner = LevelPruner(combinedmodel, config_list, combinedoptimizer)
    combinedmodel = pruner.compress()
    combinemodel_dict = combinedmodel.state_dict()
    
#     for epoch in range(10):
#         print('# Epoch {} #'.format(epoch+1))
#         train(combinedmodel, device, finetune_train_loader, combinedoptimizer, criterion)
#         pruner.update_epoch(epoch)
        
    for key in combinemodel_dict.keys():
        if 'wm' in key:
            if 'mask' in key:
                combinemodel_dict[key] = torch.ones_like(combinemodel_dict[key])

    combinedmodel.load_state_dict(combinemodel_dict)

    test_original(combinedmodel, criterion, test_loader, device)
    test_private(combinedmodel, criterion, wm_testloader, device) 
    print('###########################################################################################')


Marked model accuracy 92.53
Pruning rate: 10.0%
Testing: epoch: 1 | loss: 0.537 | Acc: 92.520
Private model testing: epoch: 1 | loss: 0.007 | Acc: 99.733
###########################################################################################
Marked model accuracy 92.53
Pruning rate: 20.0%
Testing: epoch: 1 | loss: 0.539 | Acc: 92.210
Private model testing: epoch: 1 | loss: 0.008 | Acc: 99.653
###########################################################################################
Marked model accuracy 92.53
Pruning rate: 30.0%
Testing: epoch: 1 | loss: 0.568 | Acc: 91.730
Private model testing: epoch: 1 | loss: 0.014 | Acc: 99.627
###########################################################################################
Marked model accuracy 92.53
Pruning rate: 40.0%
Testing: epoch: 1 | loss: 0.678 | Acc: 90.160
Private model testing: epoch: 1 | loss: 0.368 | Acc: 89.813
###########################################################################################
Marked model acc